<a href="https://colab.research.google.com/github/CAPost0110/IBM_Data_Science_Capstone/blob/main/Capstone_Dashboard_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
# Import required libraries
import pandas as pd
import dash
import dash.html as html
import dash.dcc as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
        style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}
            ),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                               

                                  # Add a division
          html.Div([
                                          # Create a division for adding dropdown helper text for report type
              html.Div([
                  html.H2('Choose Launch Site:', style={'margin-right': '2em'}),]
                                          ),
              # Create Dropdown     
              dcc.Dropdown(id='site-dropdown',
                  options=[
                      {'label': 'All Sites', 'value': 'ALL'},
                      {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                      {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                      {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                      {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                          ], 
                  value='ALL',        
                  placeholder="Select a Launch Site here",
                  searchable=True,
                  style={'width':'80%', 'padding':'3px', 'font-size':'20px','text-align-last':'center'})
          ]),
          html.Br(),

          html.Div(dcc.Graph(id='success-pie-chart')),
          html.Br(),

          html.P("Payload range (Kg):"),

          dcc.RangeSlider(id='payload-slider',
                          min=0, max=10000, step=1000,
                          marks={0: '0', 2500:'2500', 5000:'5000', 7500:'7500',
                                10000: '10000'},
                          value=[min_payload, max_payload]),
          html.Div(dcc.Graph(id='success-payload-scatter-chart')),
          ])


# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

# Function decorator to specify function input and output

@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(site_dropdown):
    # Select data
    site_df =  spacex_df.loc[spacex_df['Launch Site']== site_dropdown]
    if site_dropdown == 'ALL':
        fig = px.pie(spacex_df, values='class', 
        names='Launch Site', 
        title='Total Launches for All Sites')
        return fig
    else:
        fig = px.pie(site_df, names='class',title='Site Launch Total')
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [
      Input(component_id='site-dropdown', component_property='value'),
      Input(component_id='payload-slider', component_property='value')
    ]
    )


def change_graph(site_dropdown, payload_slider):
    # Select data
    site_df =  spacex_df.loc[spacex_df['Launch Site']== site_dropdown]
    payload_df =  spacex_df[(spacex_df['Payload Mass (kg)']>= payload_slider[0])&(spacex_df['Payload Mass (kg)']<=payload_slider[1])]
    site_payload_df = site_df[(site_df['Payload Mass (kg)']>= payload_slider[0])&(site_df['Payload Mass (kg)']<=payload_slider[1])]
    if site_dropdown == 'ALL':
      fig1 = px.scatter(data_frame=payload_df, x="Payload Mass (kg)", y="class", color="Booster Version Category", title="Overall Payload Success")
      return fig1
    else:
        fig1 = px.scatter(data_frame=site_payload_df, x="Payload Mass (kg)", y="class", color="Booster Version Category", title="Site Payload Success")
        return fig1


# Run the app
if __name__ == '__main__':
    app.run_server()